In [1]:
###
# import packages
###
import os
import pickle

import pandas as pd
import numpy as np
#import matplotlib.pyplot as pl
#from matplotlib import dates
import seaborn as sns
from datetime import datetime
from datetime import date
%pylab inline


Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [ ]:
#collect number of files
for file in os.listdir('Data\.'):
    if file.endswith("*.csv"):
        print(file)

In [ ]:
import glob, os   
path = r'C:\Users\talko\OneDrive - Hanzehogeschool Groningen\HanzeFitBit\Analyse\Github\hanzefit-ml\Data'                # use your own path
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)


In [ ]:
# put candidate files in a dataframe
df = concatenated_df

In [ ]:
#add Date_time column
df['mdate'] = pd.to_datetime(df['date'])
#df.dtypes

In [ ]:
#treatment_id's
treatment_id=df['treatment_id'].unique()
#print(treatment_id)

In [ ]:
df['year'] = df['mdate'].dt.year
df['week'] = df['mdate'].dt.week
df['weekday']=df['mdate'].dt.weekday
df['hour'] = df['mdate'].dt.hour

In [ ]:
for num in treatment_id:
     globals()['df%s' % num]=df[(df.treatment_id==num)]
        

In [ ]:
#group by  use resample
for num in treatment_id:
    globals()['df_h_%s'% num] = globals()['df%s' %num].iloc[:,[6,8]]
    globals()['df_h_%s'% num] = globals()['df_h_%s'% num].set_index('mdate')
    globals()['df_h_m%s' % num] = globals()['df_h_%s'% num].resample('H').sum()
#year_hour_means['treatment_id']=df1['treatment_id']

In [ ]:
#add treatment id to the grouped dataframes
for num in treatment_id:
    globals()['df_h_m%s'% num] = globals()['df_h_m%s'% num].reset_index()
    globals()['df_h_m%s'% num]['treatment_id']= num
    globals()['df_h_m%s'% num]['year'] = globals()['df_h_m%s'% num]['mdate'].dt.year
    globals()['df_h_m%s'% num]['week'] = globals()['df_h_m%s'% num]['mdate'].dt.week
    globals()['df_h_m%s'% num]['weekday'] = globals()['df_h_m%s'% num]['mdate'].dt.weekday
    globals()['df_h_m%s'% num]['hour'] = globals()['df_h_m%s'% num]['mdate'].dt.hour
    globals()['df_h_m%s'% num]['date'] = globals()['df_h_m%s'% num]['mdate'].dt.date
 

In [ ]:
#concatenate one dataframe again
loop_indicator=0
for num in treatment_id:
        if loop_indicator==0:
            concatenate_df=globals()['df_h_m%s'% num]
            loop_indicator=1
        else:    
            concatenate_df =concatenate_df.append(globals()['df_h_m%s'% num])
        


In [ ]:
#remove empty steps
concatenate_df['steps'].replace('',np.nan, inplace=True)
concatenate_df.dropna(subset=['steps'], inplace=True)


In [ ]:
#export data to database
import cx_Oracle
con = cx_Oracle.connect('hanzefitbit/hanzefitbit@127.0.0.1/xe')


In [ ]:
for i in treatment_id:
    #numpy.int64 is not accepted by the cx_oracle interface thats the reason for changing the format in a float
    i_i=i.astype(float)
    cur = con.cursor()
    sql = "INSERT INTO HFT_TREATMENT_T(TREATMENT_ID) VALUES (:1)"
    cur.execute(sql,(i_i,))
    con.commit()
cur.close()    

In [ ]:
cur = con.cursor()
rows = [tuple(x) for x in concatenate_df.values]
cur.prepare("BEGIN hft_insert_into_hft_data_t_p(:1,:2,:3,:4,:5,:6,:7,:8);END;")
cur.executemany(None,rows)
con.commit()
cur.close()
#print(rows)